<a href="https://colab.research.google.com/github/aitorvv96/cajon_desastre/blob/master/Python/manage_tasks_on_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Earth Engine basics**
Aitor Vázquez Veloso, June 2022  
More info [here](https://github.com/aitorvv96/cajon_desastre)

# **Authentication on Google Earth Engine**

In [ ]:
# import library
import ee

In [ ]:
# Authentication and initialization of GEE
ee.Authenticate() 
ee.Initialize() 

# **Download images without exceeding the tasks limit**
  
This code is important to manage the tasks sent to the function that downloads the images to not overpass the limits and stop the process.
  
To use the following code, you have to take into account:
- *df_list* must contain a list of data with the coordinates of the point you want to obtain the image. If not, delete the first 'for' loop and put the data of the variable 'my_df'
- the next part of the code is adapted to a general structure that you can check on that [file](https://github.com/aitorvv96/cajon_desastre/blob/master/Python/get_GEE_images.ipynb)
- *managing_tasks* is the part that really matters about this code. With this, the code will be checking how many tasks are waiting to be done, and when the tasks will be finished it automatically sends more tasks to the download images function

In [ ]:
# That code will download all the df of the list without exceed the
# maximum of tasks on the GEE queue (3000)

for df in df_list:  # for each df on the list...

  my_df = pd.read_csv(df)

  # download original image
  download_MDT_images(my_df, Prefix_original_points_folder_name)  

  # download random images
  for n in range(number_of_creations):
      new_df = generate_random_point_from_csv(my_df, maximum_distance)
      csv_name = output_csv_base_name + str(n + 1) + '.csv'
      new_df.to_csv(csv_name) #new csv files will be saved in working directory
      download_MDT_images(new_df, Prefix_random_points_folder_name + str(n + 1))


  # managing tasks 
  task_list = ee.data.getTaskList()  # tasks list must be updated with each df

  while task_list[0]['state'] != 'COMPLETED':  # check if the last was downloaded

    time.sleep(60)  # wait some time to download the images
    task_list = ee.data.getTaskList()  # tasks list must be updated on each try

    if task_list[0]['state'] == 'COMPLETED':  # condition to go out of "while"

      print('-------------------')
      print('df fully downloaded')
      print('-------------------')

      break

print('ALL THE MDT DOWNLOADS HAVE FINISHED')